In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('raw_data/data.csv', sep=';', thousands='.', decimal=',', encoding='latin1')
df_gas_stations = pd.read_csv('raw_data/Gast_stations por CCAA, prov y tipo_carb.csv')
df_road_length = pd.read_csv('raw_data/Long (km)  por CCAA, PRO, title y kind_road.csv')
df_acc_vic_capro=pd.read_csv('raw_data/AA Y VV C CCAA-PRO.csv', delimiter=',', thousands='.', decimal=',')
df2=pd.read_csv('raw_data/AA.VV. RCE VIA-CCAA-PRO.csv', delimiter=',', thousands='.', decimal=',')
df3=pd.read_csv('raw_data/VV.MM ACC RCE VIA-CCAA-PRO.csv', delimiter=',', thousands='.', decimal=',')
df4=pd.read_csv('raw_data/VV.MM.HH.HH ACC C INTERYURB VEH.csv', delimiter=',', thousands='.', decimal=',')

## POPULATION DATAFRAME CLEANING AND FORMATTING

In [3]:
df.head(5)

,Provincias,Edad (grupos quinquenales),Españoles/Extranjeros,Sexo,Año,Total
0,TOTAL ESPAÑA,20-24 años,TOTAL,Ambos sexos,2022,2416033
1,TOTAL ESPAÑA,20-24 años,TOTAL,Ambos sexos,2021,2382448
2,TOTAL ESPAÑA,20-24 años,TOTAL,Ambos sexos,2020,2360667
3,TOTAL ESPAÑA,20-24 años,TOTAL,Ambos sexos,2019,2305303
4,TOTAL ESPAÑA,20-24 años,TOTAL,Ambos sexos,2018,2288096


In [4]:
print(df.shape)
print(df.describe())

(11925, 6)
                Año         Total
count  11925.000000  1.192500e+04
mean    2010.000000  1.180511e+05
std        7.211405  4.328456e+05
min     1998.000000  2.221000e+03
25%     2004.000000  2.076300e+04
50%     2010.000000  3.942100e+04
75%     2016.000000  7.090700e+04
max     2022.000000  4.190045e+06


In [5]:
# Agrupar por 'Año' y 'Provincias' y sumar los valores de 'Total'
df_grouped = df.groupby(['Año', 'Provincias'])['Total'].sum().reset_index()

In [6]:
# Eliminar las filas que contienen "TOTAL" en la columna 'Provincias'
df_grouped_filtered = df_grouped[~df_grouped['Provincias'].str.contains('TOTAL')]

In [7]:
# Eliminar los números del comienzo del nombre de las provincias usando .loc para evitar SettingWithCopyWarning
df_grouped_filtered.loc[:, 'Provincias'] = df_grouped_filtered['Provincias'].str.replace(r'^\d+\s+', '', regex=True)

In [8]:
# Renombrar las columnas usando .loc para evitar SettingWithCopyWarning
df_grouped_filtered = df_grouped_filtered.rename(columns={'Provincias': 'province', 'Año': 'year', 'Total': 'population'})

In [9]:
# Mostrar los valores únicos de la columna 'Provincias'
unique_provincias = df_grouped_filtered['province'].unique()
print(unique_provincias)

['Araba/Álava' 'Albacete' 'Alicante/Alacant' 'Almería' 'Ávila' 'Badajoz'
 'Balears, Illes' 'Barcelona' 'Burgos' 'Cáceres' 'Cádiz'
 'Castellón/Castelló' 'Ciudad Real' 'Córdoba' 'Coruña, A' 'Cuenca'
 'Girona' 'Granada' 'Guadalajara' 'Gipuzkoa' 'Huelva' 'Huesca' 'Jaén'
 'León' 'Lleida' 'Rioja, La' 'Lugo' 'Madrid' 'Málaga' 'Murcia' 'Navarra'
 'Ourense' 'Asturias' 'Palencia' 'Palmas, Las' 'Pontevedra' 'Salamanca'
 'Santa Cruz de Tenerife' 'Cantabria' 'Segovia' 'Sevilla' 'Soria'
 'Tarragona' 'Teruel' 'Toledo' 'Valencia/València' 'Valladolid' 'Bizkaia'
 'Zamora' 'Zaragoza' 'Ceuta' 'Melilla']


In [10]:
# Crear un diccionario de mapeo para las provincias
provincias_map = {
    'Araba/Álava': 'Álava',
    'Alicante/Alacant': 'Alicante',
    'Balears, Illes': 'Baleares',
    'Castellón/Castelló': 'Castellón',
    'Girona': 'Gerona',
    'Gipuzkoa': 'Guipúzcoa',
    'Lleida': 'Lérida',
    'Ourense': 'Orense',
    'Santa Cruz de Tenerife': 'Tenerife, S.C.',
    'Valencia/València': 'Valencia',
    'Bizkaia': 'Vizcaya'
}

# Aplicar el mapeo a la columna 'province'
df_grouped_filtered['province'] = df_grouped_filtered['province'].replace(provincias_map)

In [11]:
# Crear un diccionario de mapeo para las provincias y sus comunidades autónomas
region_map = {
    'Álava': 'País Vasco',
    'Albacete': 'Castilla-La Mancha',
    'Alicante': 'C. Valenciana',
    'Almería': 'Andalucía',
    'Ávila': 'Castilla y León',
    'Badajoz': 'Extremadura',
    'Baleares': 'Islas Baleares',
    'Barcelona': 'Cataluña',
    'Burgos': 'Castilla y León',
    'Cáceres': 'Extremadura',
    'Cádiz': 'Andalucía',
    'Castellón': 'C. Valenciana',
    'Ciudad Real': 'Castilla-La Mancha',
    'Córdoba': 'Andalucía',
    'Coruña, A': 'Galicia',
    'Cuenca': 'Castilla-La Mancha',
    'Gerona': 'Cataluña',
    'Granada': 'Andalucía',
    'Guadalajara': 'Castilla-La Mancha',
    'Guipúzcoa': 'País Vasco',
    'Huelva': 'Andalucía',
    'Huesca': 'Aragón',
    'Jaén': 'Andalucía',
    'León': 'Castilla y León',
    'Lérida': 'Cataluña',
    'Rioja, La': 'La Rioja',
    'Lugo': 'Galicia',
    'Madrid': 'Madrid',
    'Málaga': 'Andalucía',
    'Murcia': 'Murcia',
    'Navarra': 'Navarra',
    'Orense': 'Galicia',
    'Asturias': 'Asturias',
    'Palencia': 'Castilla y León',
    'Palmas, Las': 'Canarias',
    'Pontevedra': 'Galicia',
    'Salamanca': 'Castilla y León',
    'Tenerife, S.C.': 'Canarias',
    'Cantabria': 'Cantabria',
    'Segovia': 'Castilla y León',
    'Sevilla': 'Andalucía',
    'Soria': 'Castilla y León',
    'Tarragona': 'Cataluña',
    'Teruel': 'Aragón',
    'Toledo': 'Castilla-La Mancha',
    'Valencia': 'C. Valenciana',
    'Valladolid': 'Castilla y León',
    'Vizcaya': 'País Vasco',
    'Zamora': 'Castilla y León',
    'Zaragoza': 'Aragón',
    'Ceuta': 'Ceuta',
    'Melilla': 'Melilla'
}

# Crear la nueva columna 'region' usando el diccionario de mapeo
df_grouped_filtered.insert(1, 'region', df_grouped_filtered['province'].map(region_map))

In [12]:
# Mostrar las filas que contienen valores nulos
df_grouped_filtered[df_grouped_filtered.isnull().any(axis=1)]

,year,region,province,population


In [13]:
df_pop_prov = df_grouped_filtered

In [14]:
df_pop_prov

,year,region,province,population
0,1998,País Vasco,Álava,186185
1,1998,Castilla-La Mancha,Albacete,207595
2,1998,C. Valenciana,Alicante,826576
3,1998,Andalucía,Almería,299675
4,1998,Castilla y León,Ávila,93767
...,...,...,...,...
1319,2022,País Vasco,Vizcaya,680078
1320,2022,Castilla y León,Zamora,92999
1321,2022,Aragón,Zaragoza,576511
1322,2022,Ceuta,Ceuta,51147


## GAS STATIONS DATAFRAME LOADING AND FORMATTING

In [15]:
df_gas_stations.head(5)

,year,comunidad,provincia,Carburante,cantidad,Textbox4,Textbox6,Textbox10
0,2013,Andalucía,Almería,Gasolina 95 E5,NaN,NaN,Total 2013,NaN
1,2013,Andalucía,Almería,Gasolina 95 E10,NaN,NaN,Total 2013,NaN
2,2013,Andalucía,Almería,Gasolina 95 E5 Premium,NaN,NaN,Total 2013,NaN
3,2013,Andalucía,Almería,Gasolina 95,152.0,1.548,Total 2013,8.744
4,2013,Andalucía,Almería,Gasolina 95 (G.protección),NaN,NaN,Total 2013,NaN


In [16]:
print(df_gas_stations.shape)
print(df_gas_stations.describe)

(10296, 8)
<bound method NDFrame.describe of        year  comunidad  provincia                   Carburante  cantidad  \
0      2013  Andalucía    Almería               Gasolina 95 E5       NaN   
1      2013  Andalucía    Almería              Gasolina 95 E10       NaN   
2      2013  Andalucía    Almería       Gasolina 95 E5 Premium       NaN   
3      2013  Andalucía    Almería                  Gasolina 95     152.0   
4      2013  Andalucía    Almería   Gasolina 95 (G.protección)       NaN   
...     ...        ...        ...                          ...       ...   
10291  2023  Rioja, La  Rioja, La                    Biodiésel       NaN   
10292  2023  Rioja, La  Rioja, La                    Bioetanol       NaN   
10293  2023  Rioja, La  Rioja, La  Gases licuados del petróleo       5.0   
10294  2023  Rioja, La  Rioja, La       Gas natural comprimido       2.0   
10295  2023  Rioja, La  Rioja, La          Gas natural licuado       NaN   

       Textbox4    Textbox6  Textbox10  
0

In [17]:
# Definir las columnas a eliminar
columnas_a_eliminar = ['Textbox4', 'Textbox6', 'Textbox10']

# Eliminar las columnas innecesarias
df_gas_stations = df_gas_stations.drop(columns=columnas_a_eliminar)

# Mostrar las primeras filas del DataFrame resultante
df_gas_stations.head()

,year,comunidad,provincia,Carburante,cantidad
0,2013,Andalucía,Almería,Gasolina 95 E5,NaN
1,2013,Andalucía,Almería,Gasolina 95 E10,NaN
2,2013,Andalucía,Almería,Gasolina 95 E5 Premium,NaN
3,2013,Andalucía,Almería,Gasolina 95,152.0
4,2013,Andalucía,Almería,Gasolina 95 (G.protección),NaN


In [18]:
valores_unicos_carburante = df_gas_stations['Carburante'].unique()
print(valores_unicos_carburante)

['Gasolina 95 E5' 'Gasolina 95 E10' 'Gasolina 95 E5 Premium' 'Gasolina 95'
 'Gasolina 95 (G.protección)' 'Gasolina 98 E5' 'Gasolina 98 E10'
 'Gasolina 98' 'Gasóleo A habitual' 'Gasóleo Premium' 'Nuevo gasóleo A'
 'Gasóleo B' 'Gasóleo C' 'Biodiésel' 'Bioetanol'
 'Gases licuados del petróleo' 'Gas natural comprimido'
 'Gas natural licuado']


In [19]:
# Definir los valores a eliminar
valores_a_eliminar = [
    'Gasolina 95 E5', 'Gasolina 95 E10', 'Gasolina 95 E5 Premium', 
    'Gasolina 95 (G.protección)', 'Gasolina 98 E5', 'Gasolina 98 E10', 
    'Gasóleo Premium', 'Nuevo gasóleo A', 'Biodiésel', 'Bioetanol', 
    'Gases licuados del petróleo', 'Gas natural comprimido', 'Gas natural licuado'
]

# Eliminar las filas que contienen los valores especificados en la columna 'Carburante'
df_gas_stations = df_gas_stations[~df_gas_stations['Carburante'].isin(valores_a_eliminar)]

# Mostrar las primeras filas del DataFrame resultante
df_gas_stations.head()

,year,comunidad,provincia,Carburante,cantidad
3,2013,Andalucía,Almería,Gasolina 95,152.0
7,2013,Andalucía,Almería,Gasolina 98,107.0
8,2013,Andalucía,Almería,Gasóleo A habitual,162.0
11,2013,Andalucía,Almería,Gasóleo B,66.0
12,2013,Andalucía,Almería,Gasóleo C,1.0


In [20]:
# Definir el diccionario de reemplazo
reemplazos = {
    'Gasolina 95': '95',
    'Gasolina 98': '98',
    'Gasóleo A habitual': 'A',
    'Gasóleo B': 'B',
    'Gasóleo C': 'C'
}

# Reemplazar los valores en la columna 'Carburante'
df_gas_stations['Carburante'] = df_gas_stations['Carburante'].replace(reemplazos)

# Mostrar las primeras filas del DataFrame resultante
df_gas_stations.head()

,year,comunidad,provincia,Carburante,cantidad
3,2013,Andalucía,Almería,95,152.0
7,2013,Andalucía,Almería,98,107.0
8,2013,Andalucía,Almería,A,162.0
11,2013,Andalucía,Almería,B,66.0
12,2013,Andalucía,Almería,C,1.0


In [21]:
# Rellenar los valores nulos en la columna 'cantidad' con 0
df_gas_stations['cantidad'] = df_gas_stations['cantidad'].fillna(0)

# Convertir la columna 'cantidad' a tipo int
df_gas_stations['cantidad'] = df_gas_stations['cantidad'].astype(int)

# Mostrar las primeras filas del DataFrame resultante
df_gas_stations.head()

,year,comunidad,provincia,Carburante,cantidad
3,2013,Andalucía,Almería,95,152
7,2013,Andalucía,Almería,98,107
8,2013,Andalucía,Almería,A,162
11,2013,Andalucía,Almería,B,66
12,2013,Andalucía,Almería,C,1


In [22]:
# Eliminar las filas que contienen el valor 'C' en la columna 'Carburante'
df_gas_stations = df_gas_stations[df_gas_stations['Carburante'] != 'C']

# Mostrar las primeras filas del DataFrame resultante
df_gas_stations.head()

,year,comunidad,provincia,Carburante,cantidad
3,2013,Andalucía,Almería,95,152
7,2013,Andalucía,Almería,98,107
8,2013,Andalucía,Almería,A,162
11,2013,Andalucía,Almería,B,66
21,2013,Andalucía,Cádiz,95,178


In [23]:
# Renombrar las columnas
df_gas_stations = df_gas_stations.rename(columns={
    'comunidad': 'region',
    'provincia': 'province',
    'Carburante': 'kind_gas',
    'cantidad': 'number'
})

# Mostrar las primeras filas del DataFrame resultante
df_gas_stations.head()

,year,region,province,kind_gas,number
3,2013,Andalucía,Almería,95,152
7,2013,Andalucía,Almería,98,107
8,2013,Andalucía,Almería,A,162
11,2013,Andalucía,Almería,B,66
21,2013,Andalucía,Cádiz,95,178


In [24]:
df_gas_pivot = df_gas_stations.pivot_table(index=['year', 'region', 'province'], columns='kind_gas', values='number', aggfunc='first').reset_index()
df_gas_pivot.head()

kind_gas,year,region,province,95,98,A,B
0,2013,Andalucía,Almería,152,107,162,66
1,2013,Andalucía,Cádiz,178,143,190,49
2,2013,Andalucía,Córdoba,162,97,207,128
3,2013,Andalucía,Granada,211,164,226,77
4,2013,Andalucía,Huelva,106,66,120,53


## ROADS LENGTH AND OWNERS

In [25]:
df_road_length.head(5)

,comunidad,provincia,year,Titularidad,Clase_via,tipo_via,longitud,Textbox24,Textbox25,Textbox14,...,Textbox5,Textbox10,Textbox11,Textbox26,Textbox12,Textbox13,Textbox110,Textbox27,Textbox8,Textbox6
0,Andalucía,Almería,2012,Estado,Gran capacidad,Autopistas de peaje,"28,19",Subtotal gran capacidad,"233,62",Subtotal Estado,...,"2.368,19",Subtotal Andalucía,"241,02","1.772,49","3.376,62","23.591,73","2.538,44","11.534,73","26.037,86","165.594,84"
1,Andalucía,Almería,2012,Estado,Gran capacidad,Autopistas libres y autovías,"202,74",Subtotal gran capacidad,"233,62",Subtotal Estado,...,"2.368,19",Subtotal Andalucía,"1.406,47","1.772,49","3.376,62","23.591,73","8.388,96","11.534,73","26.037,86","165.594,84"
2,Andalucía,Almería,2012,Estado,Gran capacidad,Carreteras multicarril,"2,69",Subtotal gran capacidad,"233,62",Subtotal Estado,...,"2.368,19",Subtotal Andalucía,"125,01","1.772,49","3.376,62","23.591,73","607,34","11.534,73","26.037,86","165.594,84"
3,Andalucía,Almería,2012,Estado,Resto de vías,Carreteras convencionales,"136,26",Subtotal resto de vías,"136,26",Subtotal Estado,...,"2.368,19",Subtotal Andalucía,"1.604,13","1.604,13","3.376,62","23.591,73","14.503,12","14.503,12","26.037,86","165.594,84"
4,Andalucía,Almería,2012,Comunidades autónomas,Gran capacidad,Autopistas de peaje,NaN,Subtotal gran capacidad,"103,89",Subtotal comunidades autónomas,...,"2.368,19",Subtotal Andalucía,NaN,"920,80","10.415,73","23.591,73","322,11","3.739,74","71.381,31","165.594,84"


In [26]:
print(df_road_length.shape)
print(df_road_length.describe)

(6864, 22)
<bound method NDFrame.describe of       comunidad  provincia  year              Titularidad       Clase_via  \
0     Andalucía    Almería  2012                   Estado  Gran capacidad   
1     Andalucía    Almería  2012                   Estado  Gran capacidad   
2     Andalucía    Almería  2012                   Estado  Gran capacidad   
3     Andalucía    Almería  2012                   Estado   Resto de vías   
4     Andalucía    Almería  2012    Comunidades autónomas  Gran capacidad   
...         ...        ...   ...                      ...             ...   
6859  Rioja, La  Rioja, La  2022    Comunidades autónomas   Resto de vías   
6860  Rioja, La  Rioja, La  2022  Diputaciones y cabildos  Gran capacidad   
6861  Rioja, La  Rioja, La  2022  Diputaciones y cabildos  Gran capacidad   
6862  Rioja, La  Rioja, La  2022  Diputaciones y cabildos  Gran capacidad   
6863  Rioja, La  Rioja, La  2022  Diputaciones y cabildos   Resto de vías   

                          tipo

In [27]:
# Definir las columnas a eliminar
columnas_a_eliminar_road = ['Textbox24', 'Textbox25', 'Textbox14', 'Textbox7', 'Textbox3', 'Textbox5', 'Textbox10', 'Textbox11', 'Textbox26', 'Textbox12', 'Textbox13', 'Textbox110', 'Textbox27', 'Textbox8', 'Textbox6']

# Eliminar las columnas innecesarias
df_road_length = df_road_length.drop(columns=columnas_a_eliminar_road)

# Mostrar las primeras filas del DataFrame resultante
df_road_length.head()

,comunidad,provincia,year,Titularidad,Clase_via,tipo_via,longitud
0,Andalucía,Almería,2012,Estado,Gran capacidad,Autopistas de peaje,"28,19"
1,Andalucía,Almería,2012,Estado,Gran capacidad,Autopistas libres y autovías,"202,74"
2,Andalucía,Almería,2012,Estado,Gran capacidad,Carreteras multicarril,"2,69"
3,Andalucía,Almería,2012,Estado,Resto de vías,Carreteras convencionales,"136,26"
4,Andalucía,Almería,2012,Comunidades autónomas,Gran capacidad,Autopistas de peaje,NaN


In [28]:
# Renombrar las columnas
df_road_length = df_road_length.rename(columns={
    'comunidad': 'region',
    'provincia': 'province',
    'Titularidad': 'owner',
    'Clase_via': 'road_size',
    'tipo_via': 'road_kind',
    'longitud': 'length'
})

# Mostrar las primeras filas del DataFrame resultante
df_road_length.head()

,region,province,year,owner,road_size,road_kind,length
0,Andalucía,Almería,2012,Estado,Gran capacidad,Autopistas de peaje,"28,19"
1,Andalucía,Almería,2012,Estado,Gran capacidad,Autopistas libres y autovías,"202,74"
2,Andalucía,Almería,2012,Estado,Gran capacidad,Carreteras multicarril,"2,69"
3,Andalucía,Almería,2012,Estado,Resto de vías,Carreteras convencionales,"136,26"
4,Andalucía,Almería,2012,Comunidades autónomas,Gran capacidad,Autopistas de peaje,NaN


In [29]:
valores_unicos_owner = df_road_length['owner'].unique()
print(valores_unicos_owner)

['Estado' 'Comunidades autónomas' 'Diputaciones y cabildos']


In [30]:
# Definir el diccionario de reemplazo
reemplazos_owner = {
    'Estado': 'State',
    'Comunidades autónomas': 'CCAA',
    'Diputaciones y cabildos': 'provincial'
}

# Reemplazar los valores en la columna 'owner'
df_road_length['owner'] = df_road_length['owner'].replace(reemplazos_owner)

# Mostrar las primeras filas del DataFrame resultante
df_road_length.head()

,region,province,year,owner,road_size,road_kind,length
0,Andalucía,Almería,2012,State,Gran capacidad,Autopistas de peaje,"28,19"
1,Andalucía,Almería,2012,State,Gran capacidad,Autopistas libres y autovías,"202,74"
2,Andalucía,Almería,2012,State,Gran capacidad,Carreteras multicarril,"2,69"
3,Andalucía,Almería,2012,State,Resto de vías,Carreteras convencionales,"136,26"
4,Andalucía,Almería,2012,CCAA,Gran capacidad,Autopistas de peaje,NaN


In [31]:
valores_unicos_road_size = df_road_length['road_size'].unique()
print(valores_unicos_road_size)

['Gran capacidad' 'Resto de vías']


In [32]:
# Definir el diccionario de reemplazo para road_size
reemplazos_road_size = {
    'Gran capacidad': 'big_size',
    'Resto de vías': 'rest'
}

# Reemplazar los valores en la columna 'road_size'
df_road_length['road_size'] = df_road_length['road_size'].replace(reemplazos_road_size)

# Mostrar las primeras filas del DataFrame resultante
df_road_length.head()

,region,province,year,owner,road_size,road_kind,length
0,Andalucía,Almería,2012,State,big_size,Autopistas de peaje,"28,19"
1,Andalucía,Almería,2012,State,big_size,Autopistas libres y autovías,"202,74"
2,Andalucía,Almería,2012,State,big_size,Carreteras multicarril,"2,69"
3,Andalucía,Almería,2012,State,rest,Carreteras convencionales,"136,26"
4,Andalucía,Almería,2012,CCAA,big_size,Autopistas de peaje,NaN


In [33]:
# Definir el diccionario de reemplazo para road_kind
reemplazos_road_kind = {
    'Autopistas de peaje': 'toll_motorways',
    'Autopistas libres y autovías': 'free_motorways',
    'Carreteras multicarril': 'multiple_road',
    'Carreteras convencionales': 'single_road'
}

# Reemplazar los valores en la columna 'road_kind'
df_road_length['road_kind'] = df_road_length['road_kind'].replace(reemplazos_road_kind)

# Mostrar las primeras filas del DataFrame resultante
df_road_length.head()

,region,province,year,owner,road_size,road_kind,length
0,Andalucía,Almería,2012,State,big_size,toll_motorways,"28,19"
1,Andalucía,Almería,2012,State,big_size,free_motorways,"202,74"
2,Andalucía,Almería,2012,State,big_size,multiple_road,"2,69"
3,Andalucía,Almería,2012,State,rest,single_road,"136,26"
4,Andalucía,Almería,2012,CCAA,big_size,toll_motorways,NaN


In [34]:
df_road_pivot = df_road_length.pivot_table(index=['year', 'region', 'province', 'owner', 'road_size'], columns='road_kind', values='length', aggfunc='first').reset_index()
df_road_pivot.head()

road_kind,year,region,province,owner,road_size,free_motorways,multiple_road,single_road,toll_motorways
0,2012,Andalucía,Almería,CCAA,big_size,"101,57","2,32",NaN,NaN
1,2012,Andalucía,Almería,CCAA,rest,NaN,NaN,"714,55",NaN
2,2012,Andalucía,Almería,State,big_size,"202,74","2,69",NaN,"28,19"
3,2012,Andalucía,Almería,State,rest,NaN,NaN,"136,26",NaN
4,2012,Andalucía,Almería,provincial,rest,NaN,NaN,"1.179,87",NaN


In [35]:
# Eliminar la columna 'road_size'
df_road_pivot = df_road_pivot.drop(columns=['road_size'])

# Mostrar las primeras filas del DataFrame resultante
df_road_pivot.head()

road_kind,year,region,province,owner,free_motorways,multiple_road,single_road,toll_motorways
0,2012,Andalucía,Almería,CCAA,"101,57","2,32",NaN,NaN
1,2012,Andalucía,Almería,CCAA,NaN,NaN,"714,55",NaN
2,2012,Andalucía,Almería,State,"202,74","2,69",NaN,"28,19"
3,2012,Andalucía,Almería,State,NaN,NaN,"136,26",NaN
4,2012,Andalucía,Almería,provincial,NaN,NaN,"1.179,87",NaN


In [36]:
# Sustituir valores nulos por 0 en df_road_pivo
df_road_pivot = df_road_pivot.fillna(0)

## ACCIDENTES CON VICTIMAS EN CARRETERA POR PROVINCIA Y CCAA

In [37]:
# Eliminar columnas innecesarias del dataframe df_acc_vic_capro
df_acc_vic_capro = df_acc_vic_capro.drop(columns=['Textbox6', 'Textbox7', 'numero1'])

In [38]:
# Renombrar las columnas del dataframe df_acc_vic_capro
df_acc_vic_capro = df_acc_vic_capro.rename(columns={
    'comunidad': 'region',
    'provincia': 'province',
    'Tipo': 'kind',
    'numero': 'number'
})

In [39]:
# Pivotar el DataFrame para convertir los valores de 'kind' en columnas
df_acc_vic_capro_pivoted = df_acc_vic_capro.pivot_table(index=['region', 'province', 'year'], columns='kind', values='number', aggfunc='sum').reset_index()

In [40]:
# Renombrar las columnas del dataframe df_acc_vic_capro_pivoted
df_acc_vic_capro_pivoted = df_acc_vic_capro_pivoted.rename(columns={
    'Heridos hospitalizados': 'hosp injuries',
    'Nº de accidentes con víctimas': 'acc with victims',
    'Total víctimas': 'total injuried',
    'Víctimas mortales': 'mortals'
})

In [41]:
# Reordenar las columnas del dataframe
df_acc_vic_capro_pivoted = df_acc_vic_capro_pivoted[['year', 'region', 'province', 'acc with victims', 'total injuried', 'mortals', 'hosp injuries']]

In [42]:
# Crear la nueva columna 'minor injuries'
df_acc_vic_capro_pivoted['minor injuries'] = df_acc_vic_capro_pivoted['total injuried'] - (df_acc_vic_capro_pivoted['mortals'] + df_acc_vic_capro_pivoted['hosp injuries'])

In [43]:
df_acc_vic_capro_pivoted

kind,year,region,province,acc with victims,total injuried,mortals,hosp injuries,minor injuries
0,2000,Andalucía,Almería,828,1597,109,390,1098
1,2001,Andalucía,Almería,904,1518,115,410,993
2,2002,Andalucía,Almería,822,1511,105,438,968
3,2003,Andalucía,Almería,850,1589,90,559,940
4,2004,Andalucía,Almería,905,1602,108,458,1036
...,...,...,...,...,...,...,...,...
1191,2018,"Rioja, La","Rioja, La",602,829,10,68,751
1192,2019,"Rioja, La","Rioja, La",594,773,18,59,696
1193,2020,"Rioja, La","Rioja, La",461,580,16,61,503
1194,2021,"Rioja, La","Rioja, La",569,731,12,70,649


## ACCIDENTES CON VICTIMAS EN LA RCE POR CCAA Y PROVINCIA

In [44]:
# Eliminar columnas innecesarias del dataframe df2
df2 = df2.drop(columns=['Textbox80', 'Textbox60', 'Textbox61', 'Textbox81', 'Textbox64', 'Textbox82'])

In [45]:
# Renombrar las columnas del dataframe df2
df2 = df2.rename(columns={
    'comunidad': 'region',
    'provincia': 'province',
    'tipo_via': 'kind_RCE_road',
    'valor': 'number'
})

In [46]:
# Modificar los valores de la columna 'kind_RCE_road'
df2['kind_RCE_road'] = df2['kind_RCE_road'].replace({
    'Autopistas de peaje': 'toll_motorways',
    'Autopistas libres y autovías': 'free_motorways',
    'Carreteras convencionales': 'simple_road'
})

In [47]:
# Pivotar el DataFrame para convertir los valores de 'kind_RCE_road' en columnas
df2_pivoted = df2.pivot_table(index=['region', 'province', 'year'], columns='kind_RCE_road', values='number', aggfunc='sum').reset_index()

In [48]:
# Cambiar los tipos de datos de las columnas float a int
df2_pivoted[['free_motorways', 'simple_road', 'toll_motorways']] = df2_pivoted[['free_motorways', 'simple_road', 'toll_motorways']].astype(int)

In [49]:
# Reordenar las columnas del dataframe df2_pivoted
df2_pivoted = df2_pivoted[['year', 'region', 'province', 'toll_motorways', 'free_motorways', 'simple_road']]

In [50]:
df2_pivoted

kind_RCE_road,year,region,province,toll_motorways,free_motorways,simple_road
0,2005,Andalucía,Almería,0,189,136
1,2006,Andalucía,Almería,0,216,107
2,2007,Andalucía,Almería,1,196,80
3,2008,Andalucía,Almería,0,143,72
4,2009,Andalucía,Almería,2,121,54
...,...,...,...,...,...,...
931,2018,"Rioja, La","Rioja, La",15,42,59
932,2019,"Rioja, La","Rioja, La",24,28,65
933,2020,"Rioja, La","Rioja, La",15,18,60
934,2021,"Rioja, La","Rioja, La",13,24,74


## VÍCTIMAS MORTALES EN LA RCE POR TIPO DE VIA, CCAA Y PROVINCIA

In [51]:
# Eliminar columnas innecesarias del dataframe df3
df3 = df3.drop(columns=['Textbox80', 'Textbox60', 'Textbox61', 'Textbox81', 'Textbox64', 'Textbox82'])

In [52]:
# Renombrar las columnas del dataframe df3
df3 = df3.rename(columns={
    'comunidad': 'region',
    'provincia': 'province',
    'valor': 'number',
    'tipo_via': 'kind_RCE_road'
})

In [53]:
# Modificar los valores de la columna 'kind_RCE_road'
df3['kind_RCE_road'] = df3['kind_RCE_road'].replace({
    'Autopistas de peaje': 'toll_motorways',
    'Autopistas libres y autovías': 'free_motorways',
    'Carreteras convencionales': 'simple_road'
})

In [54]:
# Pivotar el DataFrame para convertir los valores de 'kind_RCE_road' en columnas
df3_pivoted = df3.pivot_table(index=['region', 'province', 'year'], columns='kind_RCE_road', values='number', aggfunc='sum').reset_index()

In [55]:
# Cambiar los tipos de datos de las columnas float a int
df3_pivoted[['free_motorways', 'simple_road', 'toll_motorways']] = df3_pivoted[['free_motorways', 'simple_road', 'toll_motorways']].astype(int)

In [56]:
# Reordenar las columnas del dataframe df2_pivoted
df3_pivoted = df3_pivoted[['year', 'region', 'province', 'toll_motorways', 'free_motorways', 'simple_road']]

In [57]:
unique_provinces_df3_pivoted = df3_pivoted['province'].unique()
print(unique_provinces_df3_pivoted)

['Almería' 'Cádiz' 'Córdoba' 'Granada' 'Huelva' 'Jaén' 'Málaga' 'Sevilla'
 'Huesca' 'Teruel' 'Zaragoza' 'Asturias' 'Balears, Illes' 'Palmas, Las'
 'Santa Cruz de Tenerife' 'Cantabria' 'Albacete' 'Ciudad Real' 'Cuenca'
 'Guadalajara' 'Toledo' 'Burgos' 'León' 'Palencia' 'Salamanca' 'Segovia'
 'Soria' 'Valladolid' 'Zamora' 'Ávila' 'Barcelona' 'Girona' 'Lleida'
 'Tarragona' 'Ceuta' 'Alicante/Alacant' 'Castellón/Castelló'
 'Valencia/Valéncia' 'Badajoz' 'Cáceres' 'Coruña, A' 'Lugo' 'Ourense'
 'Pontevedra' 'Madrid' 'Melilla' 'Murcia' 'Navarra' 'Guipúzcoa' 'Vizcaya'
 'Álava' 'Rioja, La']


## VÍCTIMAS MORTALES Y HERIDOS EN EN TRAMOS URBANOS E INTERURBANOS POR TIPO DE VEHICULO.

In [58]:
# Eliminar columnas innecesarias del dataframe df4
df4 = df4.drop(columns=['Textbox22', 'Textbox19', 'Textbox23'])

In [59]:
# Renombrar las columnas del dataframe df4 para mayor claridad
df4 = df4.rename(columns={
    'infoAcciden': 'accident_info',
    'ambito': 'scope',
    'nombreVehiculo': 'vehicle_name',
    'valor': 'value'
})

# Verificar y corregir los tipos de datos de las columnas
df4['year'] = df4['year'].astype(int)
df4['value'] = df4['value'].astype(float)

# Eliminar filas duplicadas si las hay
df4 = df4.drop_duplicates()

# Manejar valores nulos en la columna 'value'
df4['value'] = df4['value'].fillna(0)

In [60]:
# Cambiar las columnas de tipo float a int en df4
df4['value'] = df4['value'].astype(int)

In [61]:
unique_accident_info = df4['accident_info'].unique()
print(unique_accident_info)

['Víctimas mortales' 'Heridos hospitalizados']


In [62]:
# Cambiar los valores de la columna 'accident_info'
df4['accident_info'] = df4['accident_info'].replace({
    'Víctimas mortales': 'mortals',
    'Heridos hospitalizados': 'hosp injuries'
})

In [63]:
unique_scope = df4['scope'].unique()
print(unique_scope)

['Vías interurbanas' 'Vías urbanas']


In [64]:
# Cambiar los valores de la columna 'scope'
df4['scope'] = df4['scope'].replace({
    'Vías interurbanas': 'interurban',
    'Vías urbanas': 'urban'
})

In [65]:
unique_vehicle_names = df4['vehicle_name'].unique()
print(unique_vehicle_names)

['Motocicletas y ciclomotores' 'Turismos' 'Camiones y furgonetas'
 'Autobuses' 'Bicicletas' 'VMP' 'Otros vehículos' 'Peatones']


In [66]:
# Cambiar los valores de la columna 'vehicle_name'
df4['vehicle_name'] = df4['vehicle_name'].replace({
    'Motocicletas y ciclomotores': 'moto',
    'Turismos': 'cars',
    'Camiones y furgonetas': 'trucks',
    'Autobuses': 'buses',
    'Bicicletas': 'bic'
})

In [67]:
# Cambiar los valores de la columna 'vehicle_name'
df4['vehicle_name'] = df4['vehicle_name'].replace({
    'VMP': 'pmv',
    'Peatones': 'pedestrian'
})

In [68]:
# Cambiar el valor 'Otros vehículos' por 'others' en la columna 'vehicle_name'
df4['vehicle_name'] = df4['vehicle_name'].replace('Otros vehículos', 'others')

In [69]:
# Renombrar las columnas del dataframe df4
df4 = df4.rename(columns={
    'accident_info': 'kind_vict',
    'scope': 'kind_road',
    'vehicle_name': 'vehic',
    'value': 'number'
})

In [70]:
df_pop = df_pop_prov
df_gasstations = df_gas_pivot
df_roads = df_road_pivot
df_accs = df_acc_vic_capro_pivoted
df_acc_vic_rec= df2_pivoted
df_vvmm_rce= df3_pivoted
df_vvmm_hh= df4

## ARMONIZACIÓN DE AÑOS

In [71]:
# Obtener los años de cada dataframe
years_df_pop = set(df_pop['year'].unique())
years_df_gasstations = set(df_gasstations['year'].unique())
years_df_roads = set(df_roads['year'].unique())
years_df_accs = set(df_accs['year'].unique())
years_df_acc_vic_rec = set(df_acc_vic_rec['year'].unique())
years_df_vvmm_rce = set(df_vvmm_rce['year'].unique())
years_df_vvmm_hh = set(df_vvmm_hh['year'].unique())

# Encontrar los años en común
common_years = years_df_pop & years_df_gasstations & years_df_roads & years_df_accs & years_df_acc_vic_rec & years_df_vvmm_rce & years_df_vvmm_hh

print(common_years)

{2016, 2017, 2018, 2019, 2020, 2021, 2022, 2013, 2014, 2015}


In [72]:
# Crear un diccionario para almacenar los rangos de años de cada dataframe
years_range = {
    'df_pop': sorted(years_df_pop),
    'df_gasstations': sorted(years_df_gasstations),
    'df_roads': sorted(years_df_roads),
    'df_accs': sorted(years_df_accs),
    'df_acc_vic_rec': sorted(years_df_acc_vic_rec),
    'df_vvmm_rce': sorted(years_df_vvmm_rce),
    'df_vvmm_hh': sorted(years_df_vvmm_hh)
}

# Mostrar los rangos de años
for df_name, years in years_range.items():
    print(f"{df_name}: {years[0]} - {years[-1]}")

df_pop: 1998 - 2022
df_gasstations: 2013 - 2023
df_roads: 2012 - 2022
df_accs: 2000 - 2022
df_acc_vic_rec: 2005 - 2022
df_vvmm_rce: 2008 - 2022
df_vvmm_hh: 2000 - 2022


In [74]:
# Definir el rango de años
start_year = 2013
end_year = 2022

# Filtrar los dataframes por el rango de años
df_pop_filtered = df_pop[(df_pop['year'] >= start_year) & (df_pop['year'] <= end_year)]
df_gasstations_filtered = df_gasstations[(df_gasstations['year'] >= start_year) & (df_gasstations['year'] <= end_year)]
df_roads_filtered = df_roads[(df_roads['year'] >= start_year) & (df_roads['year'] <= end_year)]
df_accs_filtered = df_accs[(df_accs['year'] >= start_year) & (df_accs['year'] <= end_year)]
df_acc_vic_rec_filtered = df_acc_vic_rec[(df_acc_vic_rec['year'] >= start_year) & (df_acc_vic_rec['year'] <= end_year)]
df_vvmm_rce_filtered = df_vvmm_rce[(df_vvmm_rce['year'] >= start_year) & (df_vvmm_rce['year'] <= end_year)]
df_vvmm_hh_filtered = df_vvmm_hh[(df_vvmm_hh['year'] >= start_year) & (df_vvmm_hh['year'] <= end_year)]

## PROBLEMAS CON NOMBRES COOFICIALES ANTES DE ARMONIZAR POR RAID

# Valencia/Valéncia

In [75]:
# Lista de dataframes a verificar
dataframes = [df_gasstations_filtered, df_roads_filtered, df_accs_filtered, df_acc_vic_rec_filtered, df_vvmm_rce_filtered]

# Iterar sobre cada dataframe y reemplazar el valor
for df in dataframes:
    if 'Valencia/València' in df['province'].values:
        df['province'] = df['province'].replace('Valencia/València', 'Valencia')
    if 'Valencia/Valéncia' in df['province'].values:
        df['province'] = df['province'].replace('Valencia/Valéncia', 'Valencia')

# Verificar los cambios
for df in dataframes:
    print(df['province'].unique())

['Almería' 'Cádiz' 'Córdoba' 'Granada' 'Huelva' 'Jaén' 'Málaga' 'Sevilla'
 'Huesca' 'Teruel' 'Zaragoza' 'Asturias' 'Balears, Illes' 'Palmas, Las'
 'Santa Cruz de Tenerife' 'Cantabria' 'Albacete' 'Ciudad Real' 'Cuenca'
 'Guadalajara' 'Toledo' 'Burgos' 'León' 'Palencia' 'Salamanca' 'Segovia'
 'Soria' 'Valladolid' 'Zamora' 'Ávila' 'Barcelona' 'Girona' 'Lleida'
 'Tarragona' 'Ceuta' 'Alicante/Alacant' 'Castellón/Castelló' 'Valencia'
 'Badajoz' 'Cáceres' 'Coruña, A' 'Lugo' 'Ourense' 'Pontevedra' 'Madrid'
 'Melilla' 'Murcia' 'Navarra' 'Guipúzcoa' 'Vizcaya' 'Álava' 'Rioja, La']
['Almería' 'Cádiz' 'Córdoba' 'Granada' 'Huelva' 'Jaén' 'Málaga' 'Sevilla'
 'Huesca' 'Teruel' 'Zaragoza' 'Asturias' 'Balears, Illes' 'Palmas, Las'
 'Santa Cruz de Tenerife' 'Cantabria' 'Albacete' 'Ciudad Real' 'Cuenca'
 'Guadalajara' 'Toledo' 'Burgos' 'León' 'Palencia' 'Salamanca' 'Segovia'
 'Soria' 'Valladolid' 'Zamora' 'Ávila' 'Barcelona' 'Girona' 'Lleida'
 'Tarragona' 'Ceuta' 'Alicante/Alacant' 'Castellón/Castelló' 'V

C:\Users\Carlos\AppData\Local\Temp\ipykernel_1304\1802726620.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['province'] = df['province'].replace('Valencia/Valéncia', 'Valencia')


In [76]:
# Lista de dataframes a verificar
dataframes = [df_gasstations_filtered, df_roads_filtered, df_accs_filtered, df_acc_vic_rec_filtered, df_vvmm_rce_filtered]

# Iterar sobre cada dataframe y reemplazar el valor
for df in dataframes:
    if 'Castellón/Castelló' in df['province'].values:
        df['province'] = df['province'].replace('Castellón/Castelló', 'Castellón')
    if 'Castellón/Castellò' in df['province'].values:
        df['province'] = df['province'].replace('Castellón/Castellò', 'Castellón')

# Verificar los cambios
for df in dataframes:
    print(df['province'].unique())

['Almería' 'Cádiz' 'Córdoba' 'Granada' 'Huelva' 'Jaén' 'Málaga' 'Sevilla'
 'Huesca' 'Teruel' 'Zaragoza' 'Asturias' 'Balears, Illes' 'Palmas, Las'
 'Santa Cruz de Tenerife' 'Cantabria' 'Albacete' 'Ciudad Real' 'Cuenca'
 'Guadalajara' 'Toledo' 'Burgos' 'León' 'Palencia' 'Salamanca' 'Segovia'
 'Soria' 'Valladolid' 'Zamora' 'Ávila' 'Barcelona' 'Girona' 'Lleida'
 'Tarragona' 'Ceuta' 'Alicante/Alacant' 'Castellón' 'Valencia' 'Badajoz'
 'Cáceres' 'Coruña, A' 'Lugo' 'Ourense' 'Pontevedra' 'Madrid' 'Melilla'
 'Murcia' 'Navarra' 'Guipúzcoa' 'Vizcaya' 'Álava' 'Rioja, La']
['Almería' 'Cádiz' 'Córdoba' 'Granada' 'Huelva' 'Jaén' 'Málaga' 'Sevilla'
 'Huesca' 'Teruel' 'Zaragoza' 'Asturias' 'Balears, Illes' 'Palmas, Las'
 'Santa Cruz de Tenerife' 'Cantabria' 'Albacete' 'Ciudad Real' 'Cuenca'
 'Guadalajara' 'Toledo' 'Burgos' 'León' 'Palencia' 'Salamanca' 'Segovia'
 'Soria' 'Valladolid' 'Zamora' 'Ávila' 'Barcelona' 'Girona' 'Lleida'
 'Tarragona' 'Ceuta' 'Alicante/Alacant' 'Castellón' 'Valencia' 'Badajoz'

C:\Users\Carlos\AppData\Local\Temp\ipykernel_1304\1548583621.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['province'] = df['province'].replace('Castellón/Castelló', 'Castellón')


In [77]:
# Lista de dataframes a verificar
dataframes = [df_gasstations_filtered, df_roads_filtered, df_accs_filtered, df_acc_vic_rec_filtered, df_vvmm_rce_filtered]

# Iterar sobre cada dataframe y reemplazar el valor
for df in dataframes:
    if 'Alicante/Alacant' in df['province'].values:
        df['province'] = df['province'].replace('Alicante/Alacant', 'Alicante')

# Verificar los cambios
for df in dataframes:
    print(df['province'].unique())

['Almería' 'Cádiz' 'Córdoba' 'Granada' 'Huelva' 'Jaén' 'Málaga' 'Sevilla'
 'Huesca' 'Teruel' 'Zaragoza' 'Asturias' 'Balears, Illes' 'Palmas, Las'
 'Santa Cruz de Tenerife' 'Cantabria' 'Albacete' 'Ciudad Real' 'Cuenca'
 'Guadalajara' 'Toledo' 'Burgos' 'León' 'Palencia' 'Salamanca' 'Segovia'
 'Soria' 'Valladolid' 'Zamora' 'Ávila' 'Barcelona' 'Girona' 'Lleida'
 'Tarragona' 'Ceuta' 'Alicante' 'Castellón' 'Valencia' 'Badajoz' 'Cáceres'
 'Coruña, A' 'Lugo' 'Ourense' 'Pontevedra' 'Madrid' 'Melilla' 'Murcia'
 'Navarra' 'Guipúzcoa' 'Vizcaya' 'Álava' 'Rioja, La']
['Almería' 'Cádiz' 'Córdoba' 'Granada' 'Huelva' 'Jaén' 'Málaga' 'Sevilla'
 'Huesca' 'Teruel' 'Zaragoza' 'Asturias' 'Balears, Illes' 'Palmas, Las'
 'Santa Cruz de Tenerife' 'Cantabria' 'Albacete' 'Ciudad Real' 'Cuenca'
 'Guadalajara' 'Toledo' 'Burgos' 'León' 'Palencia' 'Salamanca' 'Segovia'
 'Soria' 'Valladolid' 'Zamora' 'Ávila' 'Barcelona' 'Girona' 'Lleida'
 'Tarragona' 'Ceuta' 'Alicante' 'Castellón' 'Valencia' 'Badajoz' 'Cáceres'
 'Cor

C:\Users\Carlos\AppData\Local\Temp\ipykernel_1304\2773735898.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['province'] = df['province'].replace('Alicante/Alacant', 'Alicante')


## ARMONIZAR COMUNIDADES AUTÓNOMAS

In [78]:
from fuzzywuzzy import process

# Asumiendo que tienes 6 dataframes: df1, df2, df3, df4, df5, df6
# Y que df_referencia es el dataframe con los nombres correctos de la columna 'region'

# Paso 1: Extraer los nombres correctos de 'region' del dataframe de referencia
regiones_correctas = df_pop_filtered['region'].unique()

# Paso 2: Función para mapear valores de manera difusa
def mapear_difuso(valor, lista_correcta):
    # Emparejar el valor con la lista de regiones correctas
    resultado = process.extractOne(valor, lista_correcta)
    return resultado[0] if resultado else valor  # Devuelve el valor original si no hay coincidencia

# Paso 3: Aplicar el mapeo a todos los dataframes
df_gasstations_filtered['region'] =df_gasstations_filtered['region'].apply(lambda x: mapear_difuso(x, regiones_correctas))
df_roads_filtered['region'] = df_roads_filtered['region'].apply(lambda x: mapear_difuso(x, regiones_correctas))
df_accs_filtered['region'] = df_accs_filtered['region'].apply(lambda x: mapear_difuso(x, regiones_correctas))
df_acc_vic_rec_filtered['region'] = df_acc_vic_rec_filtered['region'].apply(lambda x: mapear_difuso(x, regiones_correctas))
df_vvmm_rce_filtered['region'] = df_vvmm_rce_filtered['region'].apply(lambda x: mapear_difuso(x, regiones_correctas))

# Ahora todas las columnas 'region' de los dataframes df1 a df6 tendrán los mismos valores que df_referencia


C:\Users\Carlos\AppData\Local\Temp\ipykernel_1304\1090494187.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gasstations_filtered['region'] =df_gasstations_filtered['region'].apply(lambda x: mapear_difuso(x, regiones_correctas))
C:\Users\Carlos\AppData\Local\Temp\ipykernel_1304\1090494187.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_roads_filtered['region'] = df_roads_filtered['region'].apply(lambda x: mapear_difuso(x, regiones_correctas))
C:\Users\Carlos\AppData\Local\Temp\ipykernel_1304

In [79]:
# Obtener los valores únicos de la columna "region" en cada dataframe
regions_df_pop = set(df_pop_filtered['region'].unique())
regions_df_gasstations = set(df_gasstations_filtered['region'].unique())
regions_df_roads = set(df_roads_filtered['region'].unique())
regions_df_accs = set(df_accs_filtered['region'].unique())
regions_df_acc_vic_rec = set(df_acc_vic_rec_filtered['region'].unique())
regions_df_vvmm_rce = set(df_vvmm_rce_filtered['region'].unique())

# Comprobar si todos los dataframes tienen los mismos valores en la columna "region"
all_regions_equal = (regions_df_pop == regions_df_gasstations == regions_df_roads == 
                     regions_df_accs == regions_df_acc_vic_rec == regions_df_vvmm_rce)

print("¿Todas las columnas 'region' tienen los mismos valores en todos los dataframes?:", all_regions_equal)

# Mostrar los valores únicos de la columna "region" en cada dataframe
print("Valores únicos en df_pop_filtered['region']:", regions_df_pop)
print("Valores únicos en df_gasstations_filtered['region']:", regions_df_gasstations)
print("Valores únicos en df_roads_filtered['region']:", regions_df_roads)
print("Valores únicos en df_accs_filtered['region']:", regions_df_accs)
print("Valores únicos en df_acc_vic_rec_filtered['region']:", regions_df_acc_vic_rec)
print("Valores únicos en df_vvmm_rce_filtered['region']:", regions_df_vvmm_rce)

¿Todas las columnas 'region' tienen los mismos valores en todos los dataframes?: True
Valores únicos en df_pop_filtered['region']: {'Aragón', 'Navarra', 'Madrid', 'Galicia', 'Islas Baleares', 'Murcia', 'Castilla y León', 'C. Valenciana', 'Cataluña', 'Castilla-La Mancha', 'Extremadura', 'La Rioja', 'Ceuta', 'Asturias', 'Melilla', 'Cantabria', 'Andalucía', 'Canarias', 'País Vasco'}
Valores únicos en df_gasstations_filtered['region']: {'Aragón', 'Navarra', 'Madrid', 'Galicia', 'Islas Baleares', 'Murcia', 'Castilla y León', 'C. Valenciana', 'Cataluña', 'Castilla-La Mancha', 'Extremadura', 'La Rioja', 'Ceuta', 'Asturias', 'Melilla', 'Cantabria', 'Andalucía', 'Canarias', 'País Vasco'}
Valores únicos en df_roads_filtered['region']: {'Aragón', 'Navarra', 'Madrid', 'Galicia', 'Islas Baleares', 'Murcia', 'Castilla y León', 'C. Valenciana', 'Cataluña', 'Castilla-La Mancha', 'Extremadura', 'La Rioja', 'Ceuta', 'Asturias', 'Melilla', 'Cantabria', 'Andalucía', 'Canarias', 'País Vasco'}
Valores único

## ARMONIZANDO PROVINCIAS

In [80]:
# Asumiendo que tienes 6 dataframes: df1, df2, df3, df4, df5, df6
# Y que df_referencia es el dataframe con los nombres correctos de la columna 'region'

# Paso 1: Extraer los nombres correctos de 'region' del dataframe de referencia
regiones_correctas = df_pop_filtered['province'].unique()

# Paso 2: Función para mapear valores de manera difusa
def mapear_difuso(valor, lista_correcta):
    # Emparejar el valor con la lista de regiones correctas
    resultado = process.extractOne(valor, lista_correcta)
    return resultado[0] if resultado else valor  # Devuelve el valor original si no hay coincidencia

# Paso 3: Aplicar el mapeo a todos los dataframes
df_gasstations_filtered['province'] =df_gasstations_filtered['province'].apply(lambda x: mapear_difuso(x, regiones_correctas))
df_roads_filtered['province'] = df_roads_filtered['province'].apply(lambda x: mapear_difuso(x, regiones_correctas))
df_accs_filtered['province'] = df_accs_filtered['province'].apply(lambda x: mapear_difuso(x, regiones_correctas))
df_acc_vic_rec_filtered['province'] = df_acc_vic_rec_filtered['province'].apply(lambda x: mapear_difuso(x, regiones_correctas))
df_vvmm_rce_filtered['province'] = df_vvmm_rce_filtered['province'].apply(lambda x: mapear_difuso(x, regiones_correctas))

# Ahora todas las columnas 'region' de los dataframes df1 a df6 tendrán los mismos valores que df_referencia


C:\Users\Carlos\AppData\Local\Temp\ipykernel_1304\913902234.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gasstations_filtered['province'] =df_gasstations_filtered['province'].apply(lambda x: mapear_difuso(x, regiones_correctas))
C:\Users\Carlos\AppData\Local\Temp\ipykernel_1304\913902234.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_roads_filtered['province'] = df_roads_filtered['province'].apply(lambda x: mapear_difuso(x, regiones_correctas))
C:\Users\Carlos\AppData\Local\Temp\ipykerne

In [81]:
# Obtener los valores únicos de la columna "region" en cada dataframe
regions_df_pop = set(df_pop_filtered['province'].unique())
regions_df_gasstations = set(df_gasstations_filtered['province'].unique())
regions_df_roads = set(df_roads_filtered['province'].unique())
regions_df_accs = set(df_accs_filtered['province'].unique())
regions_df_acc_vic_rec = set(df_acc_vic_rec_filtered['province'].unique())
regions_df_vvmm_rce = set(df_vvmm_rce_filtered['province'].unique())

# Comprobar si todos los dataframes tienen los mismos valores en la columna "region"
all_regions_equal = (regions_df_pop == regions_df_gasstations == regions_df_roads == 
                     regions_df_accs == regions_df_acc_vic_rec == regions_df_vvmm_rce)

print("¿Todas las columnas 'province' tienen los mismos valores en todos los dataframes?:", all_regions_equal)

# Mostrar los valores únicos de la columna "region" en cada dataframe
print("Valores únicos en df_pop_filtered['province']:", regions_df_pop)
print("Valores únicos en df_gasstations_filtered['province']:", regions_df_gasstations)
print("Valores únicos en df_roads_filtered['province']:", regions_df_roads)
print("Valores únicos en df_accs_filtered['province']:", regions_df_accs)
print("Valores únicos en df_acc_vic_rec_filtered['province']:", regions_df_acc_vic_rec)
print("Valores únicos en df_vvmm_rce_filtered['province']:", regions_df_vvmm_rce)

¿Todas las columnas 'province' tienen los mismos valores en todos los dataframes?: True
Valores únicos en df_pop_filtered['province']: {'Segovia', 'Valencia', 'Córdoba', 'Castellón', 'Alicante', 'Cádiz', 'Lugo', 'Navarra', 'Madrid', 'Málaga', 'Murcia', 'Burgos', 'Huesca', 'Baleares', 'Pontevedra', 'Albacete', 'Ávila', 'Soria', 'Valladolid', 'Jaén', 'Gerona', 'León', 'Vizcaya', 'Palmas, Las', 'Almería', 'Barcelona', 'Coruña, A', 'Tarragona', 'Álava', 'Asturias', 'Salamanca', 'Palencia', 'Guipúzcoa', 'Tenerife, S.C.', 'Teruel', 'Ceuta', 'Melilla', 'Cantabria', 'Granada', 'Sevilla', 'Zamora', 'Cáceres', 'Rioja, La', 'Badajoz', 'Toledo', 'Guadalajara', 'Huelva', 'Orense', 'Lérida', 'Zaragoza', 'Cuenca', 'Ciudad Real'}
Valores únicos en df_gasstations_filtered['province']: {'Segovia', 'Valencia', 'Córdoba', 'Castellón', 'Alicante', 'Cádiz', 'Lugo', 'Navarra', 'Madrid', 'Málaga', 'Murcia', 'Huesca', 'Burgos', 'Baleares', 'Pontevedra', 'Albacete', 'Ávila', 'Soria', 'Valladolid', 'Jaén', 'Gero

FORMATO SIMPLE DE NOMBRES

In [82]:
df_clean_pop = df_pop_filtered
df_clean_gasstations = df_gasstations_filtered
df_clean_roads = df_roads_filtered
df_clean_accs = df_accs_filtered
df_clean_accvic_rec = df_acc_vic_rec_filtered
df_clean_vvmmrce = df_vvmm_rce_filtered

In [83]:
# Guardar los dataframes en archivos CSV
df_clean_pop.to_csv('data/df_pop_filtered.csv', index=False)
df_clean_gasstations.to_csv('data/df_gasstations_filtered.csv', index=False)
df_clean_roads.to_csv('data/df_roads_filtered.csv', index=False)
df_clean_accs.to_csv('data/df_accs_filtered.csv', index=False)
df_clean_accvic_rec.to_csv('data/df_acc_vic_rec_filtered.csv', index=False)
df_clean_vvmmrce.to_csv('data/df_vvmm_rce_filtered.csv', index=False)